In [1]:
%load_ext autoreload
%autoreload 2

import requests
import msgpack
import zlib
import traceback
import time
import os
import boto3
from concurrent.futures import ThreadPoolExecutor 
import concurrent.futures 
import datetime 
import pandas as pd 
from tqdm.notebook import tqdm
import numpy as np 


from src.model import OrderBookChunksCollection, OrderBook

# import pickle 
# PRICE_DIFF_BEST_BINS = pickle.load(open("data/price_diff_best_bins.bin", "rb"))
# AMOUNT_BEST_BINS = pickle.load(open("data/amount_best_bins.bin", "rb"))
# AMOUNT_INDICES = dict(zip(AMOUNT_BEST_BINS, list(range(len(AMOUNT_BEST_BINS)))))
# PRICE_DIFF_INDICES = dict(zip(PRICE_DIFF_BEST_BINS, list(range(len(PRICE_DIFF_BEST_BINS)))))




In [7]:
order_book_col = OrderBookChunksCollection(bucket_name="btc-order-book")

In [3]:
all_keys = order_book_col.get_all_keys(
    start_date = datetime.datetime(2019, 10, 27), 
    end_date = datetime.datetime(2021, 12,22)
)

In [19]:
chunks = OrderBookChunksCollection.get_min_chunks(keys=all_keys, min_number_of_elements=20000)

In [22]:
len(chunks)

221

In [ ]:
for day in pd.date_range('2019-11-01', '2021-09-06'):
    print(d, datetime.datetime.now())
    d = str(day)[:10]
    keys = get_keys_per_day(d, s3)
    order_books = get_all_order_books_dicts(s3, keys)
    bids_dfs = dict()
    asks_dfs = dict()
    for i, key in tqdm(enumerate(keys)):   
        asks_dfs[key] = get_asks_df_for_key(order_books, key)
        bids_dfs[key] = get_bids_df_for_key(order_books, key)
    daily_tensor = create_daily_tensor(asks_dfs, bids_dfs, keys)
    np.savez_compressed(f"/tmp/{d}.bin", daily_tensor)
    np.savez_compressed(f"/tmp/{d}_keys.bin", np.array(keys))
    s3.upload_file(Filename=f"/tmp/{d}.bin.npz", Bucket='btc-datasets', Key=f'v1/{d}/order-book-tensor.npz')
    s3.upload_file(Filename=f"/tmp/{d}_keys.bin.npz", Bucket='btc-datasets', Key=f'v1/{d}/keys.npz')
    

In [ ]:
# import random
# days = [str(s)[:10] for s in pd.date_range('2020-01-01', '2021-09-03')]
# days = random.sample(days, k = 150)
# asks_dfs = []
# bids_dfs = []
# for day in days:
#     try:
#         keys = get_keys_per_day(day, s3, max_keys=1000)
#         random.shuffle(keys)
#         for k in keys[0:7]:
#             ask_df = get_asks_df_for_key(order_books, key)
#             asks_dfs.append(ask_df)
            
#             bid_df =  get_bids_df_for_key(order_books, key)
#             bids_dfs.append(bid_df)        
#         print(day)
#     except Exception as e: 
#         print(day, "error",e )
# big_asks_df = pd.concat(asks_dfs)
# big_bids_df = pd.concat(bids_dfs)

In [ ]:
# price_diff_series = pd.concat([pd.Series(big_asks_df.asks), pd.Series(1. - big_bids_df.bids)])
# amount_series = pd.concat([pd.Series(big_asks_df.amounts), pd.Series(big_bids_df.amounts)])

# pickle.dump(pd.qcut(pd.concat([price_diff_series, pd.Series(0), pd.Series(0.5)]), q=250, duplicates='drop').cat.categories, open("price_diff_best_bins.bin", "wb"))
# pickle.dump(pd.qcut(pd.concat([amount_series, pd.Series(0), pd.Series(1000000)]), q=250, duplicates='drop').cat.categories, open("amount_best_bins.bin", "wb"))

In [ ]:
prices = []
for i in range(5000):
    key = keys[i]
    price = (order_books[key][b'bids'][0][0] / 10.  + order_books[key][b'asks'][0][0] / 10. ) / 2
    prices.append(price)
for i, key in tqdm(enumerate(keys[0:5000])):   
    
    grid = plt.GridSpec(3, 1, wspace=0.4, hspace=0.3)

    plt.figure(figsize=(20,20))
    ax1 = plt.subplot(grid[0])
    ax1.imshow(daily_tensor[0, i, :], cmap='Greys', interpolation='nearest')
    ax1.set_xticks([])
    ax1.set_yticks([])
    
    ax2 = plt.subplot(grid[1])
    ax2.imshow(daily_tensor[1, i, :], cmap='Greys', interpolation='nearest')
    ax2.set_xticks([])
    ax2.set_yticks([])
    
    ax3 = plt.subplot(grid[2])
    ax3.plot(list(range(0,5000)), prices, color='grey', lw=1)
    ax3.plot(i, prices[i], 'go',  lw=10)
    ax3.set_xticks([])
    ax3.set_yticks([])
    
    plt.savefig(f"images/{str(i).zfill(5)}.png")
    plt.close()
    